In [4]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [187]:
doc = nlp('Eleanor Rigby picks up the rice in the church')

for node in doc:
    
    if node.dep_ == 'nsubj':
        
        print('subject', get_span(doc, node))
        
    if node.pos_ == 'VERB':
        
        
        prt_children = [c for c in node.children if c.dep_ == 'prt']\
        [0]
        
        if prt_children.i < node.i:
            
            s, e = prt_children.i, node.i
        else:
            s, e = node.i, prt_children.i
            
        print('verb', doc[s: e + 1])
        
    if node.dep_ == 'dobj':
        
        det_children = [c for c in node.children if c.dep_ == 'det']\
        [0]
        
        if det_children.i < node.i:
            
            s, e = det_children.i, node.i
        else:
            s, e = node.i, det_children.i
            
        print('object', doc[s: e + 1])

subject Eleanor Rigby
verb picks up
object the rice


In [191]:
%run ../script/webnlg.py

train = WebNLGCorpus.load('train')

train_1 = train.subset(ntriples=1)

In [193]:
s = train_1.sample()

In [194]:
s.ldf

,comment,idx,lid,ltext
2509,good,4_189,Id1,"The United States is the country of origin of Lippincott, Williams & Wilkins."
2510,good,4_189,Id2,"Lippincott, Williams, and Wilkins is located in the United States."
2511,good,4_189,Id3,Lippincott Williams & Wilkins are from the United States.


In [195]:
s.mdf

,idx,mtext,m_subject,m_predicate,m_object
1067,4_189,Lippincott_Williams_&_Wilkins | country | United_States,Lippincott_Williams_&_Wilkins,country,United_States


In [199]:
for text in s.ldf.ltext.values:
    
    print(text)
    doc = nlp(text)
    
    root = [node for node in doc if node.head == node][0]
    
    nodes = [root]
    
    for node in nodes:
        
        
    
    print(root)

The United States is the country of origin of Lippincott, Williams & Wilkins.
is
Lippincott, Williams, and Wilkins is located in the United States.
located
Lippincott Williams & Wilkins are from the United States.
are


# Data Alignment

In [13]:
from nltk.stem import WordNetLemmatizer

def split(spo):
    
    return spo.split(' ')

lemmatizer = WordNetLemmatizer()

def lemmatize(spo):
    
    return ' '.join((lemmatizer.lemmatize(spo_) for spo_ in split(spo)))

from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer

stemmer_ = 'lancaster'

if stemmer_ == 'lancaster':
    stemmer = LancasterStemmer()
elif stemmer == 'porter':
    stemmer = PorterStemmer()
else:
    stemmer = SnowballStemmer()

def stem(spo):
    
    return ' '.join((stemmer.stem(spo_) for spo_ in split(spo)))

c_detect_camelcase = re.compile(r'(?<=[a-z])([A-Z])')
def split_by_camelcase(spo):
    
    return c_detect_camelcase.sub(r' \1', spo)

c_chars_to_remove = re.compile(r'[_]')
def remove_unwanted_char(spo):
    
    return c_chars_to_remove.sub(' ', spo)

def to_lower(spo):
    
    return spo.lower()

from functools import reduce

spo_pipeline = [split_by_camelcase, remove_unwanted_char, to_lower, stem]
def preprocess_spo(spo):
    
    return reduce(lambda v, f: f(v), spo_pipeline, spo)
    
def preprocess_triple(triple):

    return {k:preprocess_spo(v) for k, v in triple.items()}    

lexe_pipeline = [to_lower, stem]
def preprocess_lexe(lexe):
    
    return reduce(lambda v, f: f(v), lexe_pipeline, lexe)

In [14]:
def replace_sop(m):
    
    return "<{}>".format(next((k for k, v in m.groupdict().items() if v)))

def simple_align(triple, lexe):
    
    preprocessed_triple = preprocess_triple(triple)
    
    preprocessed_lexe = preprocess_lexe(lexe)
    
    regex = '((?P<subject>{m_subject})|(?P<predicate>{m_predicate})|(?P<object>{m_object}))'.format(**preprocessed_triple)
    
    return re.compile(regex).sub(replace_sop, preprocessed_lexe)

In [15]:
c = simple_align(e.triples(kind='dict')[0], e.lexes()[0])
c

'the <object> ar the <predicate> in <subject>.'

In [53]:
c_search_spo = re.compile(r'((?P<object><object>)|(?P<predicate><predicate>)|(?P<subject><subject>))')

def count_spo(s):
    
    return [t[0] for t in c_search_spo.findall(s)]

In [63]:
e = train.sample()
c = simple_align(e.triples(kind='dict')[0], e.lexes()[0])
c, count_spo(c), e.triples()[0]

('the character, <subject>es, was <predicate> by <object>.',
 ['<subject>', '<predicate>', '<object>'],
 {'m_object': 'Len_Wein', 'm_predicate': 'creator', 'm_subject': 'Aurakles'})

In [43]:
c

'<subject> is a <object>.'

In [44]:
c_search_spo.match(c).groupdict()

{'object': None, 'predicate': None, 'subject': '<subject>'}

In [52]:
c_search_spo.findall('<subject> <object> <predicate> <subject>')

[('<subject>', '', '', '<subject>'),
 ('<object>', '<object>', '', ''),
 ('<predicate>', '', '<predicate>', ''),
 ('<subject>', '', '', '<subject>')]

# Spacy

In [51]:
import spacy
%run ../script/spacy_util.py

nlp = spacy.load('en_core_web_lg')

In [15]:
e.triples()

[{'m_object': 'Left_Ecology_Freedom',
  'm_predicate': 'leaderParty',
  'm_subject': 'Gubbio'}]

In [63]:
e = train.sample()
pprint(e.triples())

print(e.lexes()[0])
doc = nlp(e.lexes()[0])

displacy_noun_chunks(doc)
displacy_noun_chunks(doc)

[{'m_object': 'Spain', 'm_predicate': 'country', 'm_subject': 'Arròs_negre'},
 {'m_object': 'White_rice',
  'm_predicate': 'ingredient',
  'm_subject': 'Arròs_negre'}]
White rice is an ingredient of Arros negre which is a traditional dish from Spain.


In [24]:
doc_to_df(doc)

,Text,POS,Dep,Tag,Shape,Alpha,Stop,Head,Left,Right,Entity,EntIOB,Lemma
0,The,determiner,det,DT,Xxx,X,--,Freedom,The,The,,O,the
1,Left,proper noun,compound,NNP,Xxxx,X,--,Freedom,Left,Left,,O,left
2,Ecology,proper noun,compound,NNP,Xxxxx,X,--,Freedom,Ecology,Ecology,,O,ecology
3,Freedom,proper noun,nsubj,NNP,Xxxxx,X,--,are,The,Freedom,,O,freedom
4,are,verb,ROOT,VBP,xxx,X,--,are,The,.,,O,be
5,the,determiner,det,DT,xxx,X,--,party,the,the,,O,the
6,leading,verb,amod,VBG,xxxx,X,--,party,leading,leading,,O,lead
7,party,noun,attr,NN,xxxx,X,--,are,the,Gubbio,,O,party
8,in,adposition,prep,IN,xx,X,--,party,in,Gubbio,,O,in
9,Gubbio,proper noun,pobj,NNP,Xxxxx,X,--,in,Gubbio,Gubbio,GPE,B,gubbio


In [172]:
def mask_noun_chunks(doc):
    
    lexe_s = list(doc.text)
    for nc in doc.noun_chunks:

        lexe_s[nc.start_char: nc.end_char] = '#' * len(nc.__str__())
        
    return ''.join(lexe_s)

In [197]:
values = []

for _ in range(10):
    
    e = train.sample()
    
    for lexe in e.lexes():
        
        doc = nlp(lexe)
        
        values.append({
            'lexe': lexe,
            'noun_chunks': list(doc.noun_chunks),
            'masked': mask_noun_chunks(doc),
            'triples': e.triples()
        })

In [207]:
df = pd.DataFrame(values)
df.to_html('oi.html')

In [205]:
train.sample().triples()

[{'m_object': 'Switzerland',
  'm_predicate': 'country',
  'm_subject': 'Accademia_di_Architettura_di_Mendrisio'},
 {'m_object': 'Meride',
  'm_predicate': 'neighboringMunicipality',
  'm_subject': 'Mendrisio'},
 {'m_object': 'Mendrisio',
  'm_predicate': 'city',
  'm_subject': 'Accademia_di_Architettura_di_Mendrisio'},
 {'m_object': 'Johann_Schneider-Ammann',
  'm_predicate': 'leaderName',
  'm_subject': 'Switzerland'}]